In [47]:
Failure reason
No objective metrics found after running 5 training jobs. Please ensure that the custom algorithm is emitting the objective metric as defined by the regular expression provided.

SyntaxError: invalid syntax (<ipython-input-47-b3d768f13ecc>, line 1)

New API:
https://docs.aws.amazon.com/sagemaker/latest/dg/tf.html


https://github.com/aws-samples/amazon-sagemaker-script-mode/blob/master/tf-distribution-options/tf-distributed-training.ipynb
https://medium.com/radix-ai-blog/tensorflow-sagemaker-d17774417f08

https://blog.betomorrow.com/keras-in-the-cloud-with-amazon-sagemaker-67cf11fb536

https://github.com/aws/sagemaker-python-sdk/tree/v1.12.0/src/sagemaker/tensorflow#tensorflow-sagemaker-estimators-and-models

https://aws.amazon.com/blogs/machine-learning/train-and-deploy-keras-models-with-tensorflow-and-apache-mxnet-on-amazon-sagemaker/


Convert keras estimator to https://stackoverflow.com/questions/51455863/whats-the-difference-between-a-tensorflow-keras-model-and-estimator


In [1]:
import tensorflow as tf
import tensorflow.keras as keras

In [2]:
tf.__version__

'1.15.0'

In [3]:
import numpy as np
import re
import glob

# 1. Data preparation

## Read raw npz data and write shuffled data to TFRecord

In [29]:
train, valid, test = [], [], []
classes = []
count = 0
for file in glob.glob('sketchrnn_data/*npz'):
    if "full" not in file:
        data = np.load(file, encoding='bytes', allow_pickle=True)
        class_name = re.findall(r'(\w+).npz', file)[0]
        train.append(data['train'])
        valid.append(data['valid'])
        test.append(data['test'])
        classes.append(class_name)
        count += 1
        if count == 10:
            break
X_train_raw = np.concatenate(train)
X_valid_raw = np.concatenate(valid)
X_test_raw = np.concatenate(test)

In [30]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(classes)

train_mask = np.arange(700000)
np.random.shuffle(train_mask)

valid_mask = np.arange(25000)
np.random.shuffle(valid_mask)

X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train_raw, padding='post', value=99, maxlen=160)[train_mask]
X_valid = tf.keras.preprocessing.sequence.pad_sequences(X_valid_raw, padding='post', value=99, maxlen=160)[valid_mask]
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test_raw, padding='post', value=99, maxlen=160)

y_train = le.transform(np.repeat(classes, 70000))[train_mask]
y_valid = le.transform(np.repeat(classes, 2500))[valid_mask]
y_test = le.transform(np.repeat(classes, 2500))

In [31]:
len(X_valid)

25000

In [23]:
import os

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _int64_feature_list(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def convert_dataset(X, y, name, directory):

    filename = os.path.join(directory, name + '.tfrecords')
    print(f'Writing {filename}')
    
    with tf.python_io.TFRecordWriter(filename) as writer: 
        for i in range(len(y)):
                example = tf.train.Example(features=tf.train.Features(feature={
                'feature': _int64_feature_list(X[i].reshape(-1)),
                'label': _int64_feature(y[i])
                }))
                writer.write(example.SerializeToString())

In [24]:
convert_dataset(X_train, y_train, "train", "./sketchTF")
convert_dataset(X_valid, y_valid, "valid", "./sketchTF")

Writing ./sketchTF/train.tfrecords
Writing ./sketchTF/valid.tfrecords


## Safe data to TF record from files

In [46]:
# Get class names
classes = []
files = []
count = 0
for file in glob.glob('sketchrnn_data/*npz'):
    if "full" not in file:
        class_name = re.findall(r'(\w+).npz', file)[0]
        classes.append(class_name)
        files.append(file)
        count += 1
    if count == 10:
        break
files_to_int = {c: i for i, c in enumerate(files)}

In [ ]:

def convert_dataset_from_files(files, name, directory):

    filename = os.path.join(directory, name + '.tfrecords')
    print(f'Writing {filename}')
    
    with tf.python_io.TFRecordWriter(filename) as writer: 
        for file in files:
            data = np.load(file, encoding='bytes', allow_pickle=True)
            extracted_data = tf.keras.preprocessing.sequence.pad_sequences(data[name], padding='post', value=99, maxlen=160) 
            for data_point in extracted_data:
                example = tf.train.Example(features=tf.train.Features(feature={
                'feature': _int64_feature_list(data_point.reshape(-1)),
                'label': _int64_feature(files_to_int[file])
                }))
                writer.write(example.SerializeToString())

In [80]:
for mode in ['train', 'test', 'valid']:
    convert_dataset_from_files(files, mode, "./sketchTF")

Writing ./sketchTF/train.tfrecords
Writing ./sketchTF/test.tfrecords
Writing ./sketchTF/valid.tfrecords


## Read TFRecord

In [7]:
import tensorflow as tf
tf.enable_eager_execution()

features={
            'label': tf.FixedLenFeature([], tf.int64),
            'feature': tf.FixedLenFeature([480], tf.int64)
        }

for serialized_example in tf.data.TFRecordDataset(["sketchTF/train.tfrecords"]):
    parsed_example = tf.io.parse_single_example(serialized_example, features)
    break

In [11]:
tf.reshape(parsed_example['feature'], [160, 3])

<tf.Tensor: id=76, shape=(160, 3), dtype=int64, numpy=
array([[  10,   -5,    0],
       [  82,   -9,    0],
       [ 195,  -16,    0],
       [ 117,    4,    0],
       [  59,   -2,    0],
       [  42,    7,    0],
       [  27,   10,    0],
       [   8,   10,    0],
       [   4,   19,    0],
       [   1,  129,    0],
       [  -8,   17,    0],
       [  -9,    6,    0],
       [ -46,   10,    0],
       [-264,    1,    0],
       [ -85,  -12,    0],
       [ -60,  -21,    0],
       [ -19,  -10,    0],
       [ -17,  -14,    0],
       [ -14,  -16,    0],
       [  -9,  -16,    0],
       [ -16,  -73,    1],
       [ 101,   17,    0],
       [ -19,    4,    0],
       [  -4,   10,    0],
       [   3,   16,    0],
       [  11,   18,    0],
       [  16,   12,    0],
       [  13,    1,    0],
       [  17,   -9,    0],
       [  16,  -18,    0],
       [   3,  -19,    0],
       [  -6,  -20,    0],
       [ -16,  -14,    0],
       [ -18,   -4,    0],
       [ -35,   11,    0],


In [ ]:
parsed_example['feature'].reshape

## Upload to S3

In [25]:
import sagemaker
bucket = sagemaker.Session().default_bucket() # Automatically create a bucket
prefix = 'sketchTF' # Subfolder prefix
s3_url = sagemaker.Session().upload_data(path='sketchTF', 
                                         bucket=bucket, 
                                         key_prefix=prefix+'/data/10classShuffle')
print(s3_url)

s3://sagemaker-us-west-2-549525720196/sketchTF/data/10classShuffle


In [26]:
!aws s3 ls s3://sagemaker-us-west-2-549525720196/sketchTF/data/10classShuffle/

2020-01-17 17:33:57   23664843 test.tfrecords
2020-01-17 17:33:59  663005344 train.tfrecords
2020-01-17 17:34:06   23653600 valid.tfrecords


# 2. Sagmaker Training

## Old api

In [57]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

role = sagemaker.get_execution_role() 
#"ml.p3.2xlarge",
train_instance_type='ml.p3.2xlarge'      # The type of EC2 instance which will be used for training
deploy_instance_type='ml.p2.xlarge'     # The type of EC2 instance which will be used for deployment
hyperparameters={
#    "learning_rate": 1e-4,
    "n_hidden": 1,
    "n_neurons": 50
}

estimator = TensorFlow(entry_point='sketchRNN_keras.py',
                          role=role,
                          framework_version="1.12.0",               # TensorFlow's version
                          input_mode='Pipe',
                          hyperparameters=hyperparameters,
                          training_steps=1000,
                          evaluation_steps=100,
                          train_instance_count=2,                   # "The number of GPUs instances to use"
                          train_instance_type=train_instance_type
                        )

In [36]:
train_data = 's3://sagemaker-us-west-2-549525720196/sketchTF/data/10classShuffle/train.tfrecords'
eval_data = 's3://sagemaker-us-west-2-549525720196/sketchTF/data/10classShuffle/valid.tfrecords'

estimator.fit({'train': train_data, 'eval': eval_data}, logs=False)


2020-01-17 17:55:39 Starting - Starting the training job.....
2020-01-17 17:56:09 Starting - Launching requested ML instances..................
2020-01-17 17:57:41 Starting - Preparing the instances for training...........
2020-01-17 17:58:45 Downloading - Downloading input data....
2020-01-17 17:59:08 Training - Downloading the training image...
2020-01-17 17:59:28 Training - Training image download completed. Training in progress........................................................................................................................................................................................................
2020-01-17 18:16:13 Uploading - Uploading generated training model
2020-01-17 18:16:20 Completed - Training job completed


## New Api

In [44]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

# git_config = {'repo': 'https://github.com/aws-samples/amazon-sagemaker-script-mode', 'branch': 'master'}

ps_instance_type = 'ml.p3.2xlarge'
ps_instance_count = 1

model_dir = "/opt/ml/model"

distributions = {'parameter_server': {
                    'enabled': True}
                }
hyperparameters = {'epochs': 10, 'batch-size' : 128}

role = sagemaker.get_execution_role() 

estimator_ps = TensorFlow(
                       #git_config=git_config,
                       source_dir='code',
                       entry_point='main.py', 
                       base_job_name='sketchRNN',
                       role=role,
                       input_mode='Pipe',
                       framework_version='1.13',
                       py_version='py3',
                       hyperparameters=hyperparameters,
                       train_instance_count=ps_instance_count, 
                       train_instance_type=ps_instance_type,
                       model_dir=model_dir,
                       tags = [{'Key' : 'Project', 'Value' : 'cifar10'},{'Key' : 'TensorBoard', 'Value' : 'dist'}],
                       distributions=distributions)

In [37]:
train_data = 's3://sagemaker-us-west-2-549525720196/sketchTF/data/10classShuffle/train.tfrecords'
eval_data = 's3://sagemaker-us-west-2-549525720196/sketchTF/data/10classShuffle/valid.tfrecords'

estimator_ps.fit({'train': train_data, 'eval': eval_data}, logs=False)


2020-01-22 16:59:32 Starting - Starting the training job
2020-01-22 16:59:33 Starting - Launching requested ML instances.................
2020-01-22 17:01:05 Starting - Preparing the instances for training.............
2020-01-22 17:02:14 Downloading - Downloading input data
2020-01-22 17:02:21 Training - Downloading the training image...
2020-01-22 17:02:41 Training - Training image download completed. Training in progress................................................................................
2020-01-22 17:09:22 Uploading - Uploading generated training model
2020-01-22 17:09:28 Completed - Training job completed


## Plot learning curve

In [38]:
import json 
! aws s3 cp {estimator_ps.model_data} ./ps_model/model.tar.gz
! tar -xzf ./ps_model/model.tar.gz -C ./ps_model

with open('./ps_model/ps_history.p', "r") as f:
    ps_history = json.load(f)

download: s3://sagemaker-us-west-2-549525720196/sketchRNN-2020-01-22-16-59-31-334/output/model.tar.gz to ps_model/model.tar.gz


In [39]:
ps_history

{'acc': [0.27544069290161133,
  0.5592948794364929,
  0.6362179517745972,
  0.6660656929016113,
  0.6666666865348816,
  0.6891025900840759,
  0.7059294581413269,
  0.7049278616905212,
  0.7279647588729858,
  0.7261618375778198],
 'loss': [2.0889766185711593,
  1.3086259487347724,
  1.0669012115551875,
  0.988345803358616,
  0.9500429248198484,
  0.9061867136221665,
  0.8732469448676476,
  0.8524767221548618,
  0.7947041621574988,
  0.7921125720708798],
 'val_acc': [0.4314236044883728,
  0.6050347089767456,
  0.6692708134651184,
  0.65625,
  0.6944444179534912,
  0.7465277910232544,
  0.6770833134651184,
  0.6883680820465088,
  0.7265625,
  0.7491319179534912],
 'val_loss': [1.6530858278274536,
  1.1719423797395494,
  1.0111279222700331,
  0.9839720129966736,
  0.9122723076078627,
  0.7926922308074104,
  0.9757408764627244,
  0.8585047324498495,
  0.7984801663292779,
  0.7620407342910767]}

# 3. Hyperparameter tunning

In [48]:
s = "TA: 1:40 - loss: 0.8548 - acc: 0.7031#010#010#010#010#010#010#010#"

In [49]:
import re

In [57]:
r = re.findall(, s)

In [58]:
r

['0.8548']

In [69]:
from sagemaker.parameter import (
    CategoricalParameter,
    ContinuousParameter,
    IntegerParameter,
    ParameterRange,
)
from sagemaker.tuner import HyperparameterTuner
# Define objective
objective_metric_name = 'acc'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'acc',
                       'Regex': 'val_acc: ([0-9\\.]+)'}]
# Define hyperparameter ranges
hyperparameter_ranges = {
                        #'learning_rate': ContinuousParameter(0.0001, 0.01, scaling_type='Logarithmic'), 
                            #'dropout_rate': ContinuousParameter(0.3, 1.0),
                            'n_hidden': IntegerParameter(1, 4),
                            'n_neurons': IntegerParameter(10, 100),
                            #'optimizer_type': CategoricalParameter(['sgd', 'adam']),
                        }  
# Initialise Sagemaker's hyperparametertuner
tuner = HyperparameterTuner(estimator_ps,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=20,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

In [70]:
tuner.fit({'train': train_data, 'eval': eval_data}, logs=False)

# Deploy model

# Local development

# Build a classfication model

In [32]:
model = keras.models.Sequential([
tf.keras.layers.Masking(mask_value=99, input_shape=(None, 3)),
keras.layers.LSTM(50, return_sequences=True),
keras.layers.LSTM(50, return_sequences=True),
keras.layers.LSTM(50),
keras.layers.Dense(10, activation='softmax')
])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 3)           0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 50)          10800     
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 50)          20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 10)                510       
Total params: 51,710
Trainable params: 51,710
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
N=10000
model.fit(X_train[:N], y_train[:N], epochs=10, validation_data=(X_valid, y_valid))

Train on 10000 samples, validate on 25000 samples
Epoch 1/10
10000/10000 [==============================] - 576s 58ms/sample - loss: 1.4292 - acc: 0.5032 - val_loss: 1.0432 - val_acc: 0.6525
Epoch 2/10
10000/10000 [==============================] - 571s 57ms/sample - loss: 0.9262 - acc: 0.6809 - val_loss: 0.8853 - val_acc: 0.7025
Epoch 3/10
10000/10000 [==============================] - 566s 57ms/sample - loss: 0.7979 - acc: 0.7319 - val_loss: 0.8226 - val_acc: 0.7157
Epoch 4/10
10000/10000 [==============================] - 568s 57ms/sample - loss: 0.7251 - acc: 0.7525 - val_loss: 0.7786 - val_acc: 0.7371
Epoch 5/10
10000/10000 [==============================] - 569s 57ms/sample - loss: 0.6519 - acc: 0.7829 - val_loss: 0.7146 - val_acc: 0.7605
Epoch 6/10
 9984/10000 [============================>.] - ETA: 0s - loss: 0.6156 - acc: 0.7920

KeyboardInterrupt: 